In [ ]:
# from AU_labeling.au_extraction import map_au_intensity, extract_au_from_video, parse_au_intensity
# from AU_labeling.peak_frame_description import find_peak_frame, extract_frame_by_index
# from scripts.process_meld_multimodal import describe_audio_qwen
!pip install transformers_stream_generator
!pip install tiktoken
!pip install torchaudio
import os
import cv2
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForAudioClassification
import pandas as pd
import json
import torchaudio

  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=8ac7d60839eeb80cad45d2684295d2c64324c7d198accb7ad39eb217369b672a
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers_stream_generator
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 135.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-Audio-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="cuda", trust_remote_code=True).eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

audio.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-Audio-Chat:
- audio.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-Audio-Chat:
- tokenization_qwen.py
- audio.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00004-of-00009.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00006-of-00009.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [ ]:
directory_path = "/content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav"

print(os.path.abspath(directory_path))

total=0
for root, dirs, files in os.walk(directory_path):
    for file in files:
        file_path = os.path.join(root, file)
        print(f"Found file: {file_path}")

        video_id= f'{file[:-3]}mp4'
        audio_url= f"/content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav/{file[:-3]}wav"

        query = tokenizer.from_list_format([
            {'audio': audio_url}, # Either a local path or an url
            {'text': 'Describe the speaker’s vocal delivery in this audio. Focus only on how they sound, not what they are saying. Include detailed observations about: \n- Prosody: pitch, intonation, pacing, loudness, rhythm, and pauses or hesitation \n- Voice quality: breathiness, tension, harshness, smoothness, or creakiness \n- Articulation: clarity, enunciation, and any slurring, stuttering, or irregular speech patterns \nAlso describe how expressive or monotone the speaker sounds. Be as descriptive as possible and explain what these vocal features might suggest about their emotional tone, without interpreting the actual words.'},
        ])
        response, history = model.chat(tokenizer, query=query, history=None)
        print(response)



        result = {
            "video_id": video_id,
            "audio_description": response
        }

        try:
            with open(f"audio_descriptions.json", "r") as f:
                data = json.load(f)
        except FileNotFoundError:
            data = []

        data.append(result)

        with open(f"audio_descriptions.json", "w") as f:
            json.dump(data, f, indent=2)
        total+=1
        print(total)

/content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav
Found file: /content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav/dia233_utt4.wav
The speaker's vocal delivery in this audio is expressive and monotonous. Their pitch ranges from a high-pitched, almost child-like tone to a low, grumbly tone. They speak in a slow, measured pace, with occasional pauses and hesitations. Their voice quality is clear, with no noticeable roughness or strain. They enunciate their words clearly, with no slurring or stuttering. Overall, their vocal delivery suggests that they are expressing a mix of emotions, including frustration, sadness, and perhaps a hint of anger.
1
Found file: /content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav/dia279_utt14.wav
This is a man speaking, and his tone sounds negative.
2
Found file: /content/drive/MyDrive/MIT/UROPs/Fluid Interfaces Laboratory/test_subset_wav/dia239_utt7.wav
This is a woman speaking, and 